In [1]:
# !kaggle datasets download fullmetal26/glovetwitter27b100dtxt
# !unzip glovetwitter27b100dtxt.zip

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, SimpleRNN, Activation, Dropout, Conv1D
from tensorflow.keras.layers import Embedding, Flatten, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics import classification_report

In [ ]:
def load_glove_model(glove_file):
    print("[INFO]Loading GloVe Model...")
    model = {}
    with open(glove_file, 'r', encoding="utf-8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embeddings = [float(val) for val in split_line[1:]]
            model[word] = embeddings
    print("[INFO] Done...{} words loaded!".format(len(model)))
    return model
glove_model = load_glove_model("/content/glove.twitter.27B.100d.txt")

[INFO]Loading GloVe Model...
[INFO] Done...14726 words loaded!


In [ ]:
# adopted from utils.py
nlp = spacy.load("en_core_web_sm")

def remove_stopwords(sentence):
    '''
    function to remove stopwords
        input: sentence - string of sentence
    '''
    new = []
    # tokenize sentence
    sentence = nlp(sentence)
    for tk in sentence:
        if (tk.is_stop == False) & (tk.pos_ !="PUNCT"):
            new.append(tk.string.strip())
    # convert back to sentence string
    c = " ".join(str(x) for x in new)
    return c


def lemmatize(sentence):
    '''
    function to do lemmatization
        input: sentence - string of sentence
    '''
    sentence = nlp(sentence)
    s = ""
    for w in sentence:
        s +=" "+w.lemma_
    return nlp(s)

def sent_vectorizer(sent, model):
    '''
    sentence vectorizer using the pretrained glove model
    '''
    sent_vector = np.zeros(200)
    num_w = 0
    for w in sent.split():
        try:
            # add up all token vectors to a sent_vector
            sent_vector = np.add(sent_vector, model[str(w)])
            num_w += 1
        except:
            pass
    return sent_vector

In [ ]:
# !wget https://raw.githubusercontent.com/haochen23/nlp-rnn-lstm-sentiment/master/training.1600000.processed.noemoticon.csv

In [ ]:
# data = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None, encoding='latin-1')
data = pd.read_csv("/content/pre_processed_final.csv")
print(data.shape)
data = data[["sentiment", "text"]]
data = data.dropna()
data.head()

(28777, 3)


,sentiment,text
0,1.0,is lookin 4ward to a long weekend really dont...
1,1.0,myweakness is music and i live to meet the pe...
2,1.0,figured out the internet on my new ipod
3,1.0,cant wait to worship with you guys tonight itl...
4,1.0,congrats james im sure the book is going to b...


In [ ]:
data_X = data['text']
data_y = data['sentiment']
data_y = pd.get_dummies(data_y)

In [ ]:
data_y = data_y.to_numpy()
data_y

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=uint8)

In [ ]:
n = 6542
print(data['sentiment'][n], data_y[n])

-1.0 [1 0 0]


In [ ]:
max_vocab = 18000
max_len = 15
tokenizer = Tokenizer(num_words=max_vocab)

In [ ]:
data_X = data_X.to_numpy()
print(data_X)
tokenizer.fit_on_texts(data_X)
sequences = tokenizer.texts_to_sequences(data_X)
word_index = tokenizer.word_index


['is lookin 4ward to a long weekend  really dont want to go to work 2day tho  x'
 'myweakness  is music and i live to meet the people who make it'
 'figured out the internet on my new ipod' ...
 'speaking of chris jericho it is wednesday today is it not time for some catching up on wrestling'
 'im so jealous of everyone at the justin bieber concert no worries tho ill be seeing maroon 5 in march  roadtrippin'
 'trending tim tebow is now dating cave woman from 10000 bc 1st date question what was it like in a time before jesus  tebowmania']


In [ ]:
print('Found %s unique tokens.' % len(word_index))

Found 30651 unique tokens.


In [ ]:
data_keras = pad_sequences(sequences, maxlen=max_len, padding="post")
# data_keras = data_keras.reshape(data_X.shape[0], 15, 1)
data_keras.shape



(28398, 15)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(data_keras, data_y, test_size = 0.3, random_state=42)

In [ ]:
# adopted from sent_tran_eval.py
def build_model(nb_words, rnn_model="SimpleRNN", embedding_matrix=None):
    '''
    build_model function:
    inputs: 
        rnn_model - which type of RNN layer to use, choose in (SimpleRNN, LSTM, GRU)
        embedding_matrix - whether to use pretrained embeddings or not
    '''
    model = Sequential()
    # add an embedding layer
    if embedding_matrix is not None:
        model.add(Embedding(nb_words, 
                        100, 
                        weights=[embedding_matrix], 
                        input_length= max_len,
                        trainable = False))
    else:
        model.add(Embedding(nb_words, 
                        100, 
                        input_length= max_len,
                        trainable = False))
        
    # add an RNN layer according to rnn_model
    if rnn_model == "SimpleRNN":
        model.add(SimpleRNN(200))
    elif rnn_model == "LSTM":
        model.add(LSTM(200))
    else:
        model.add(GRU(200))
    # model.add(Dense(500,activation='relu'))
    # model.add(Dense(500, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', 
                optimizer='adam',
                metrics=['accuracy'])
    return model

In [ ]:
model_rnn = build_model(nb_words, "SimpleRNN", embedding_matrix)
model_rnn.fit(train_X, train_y, epochs=20, batch_size=60,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_rnn.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20
332/332 [==============================] - 9s 22ms/step - loss: 0.8797 - accuracy: 0.6029 - val_loss: 0.8347 - val_accuracy: 0.6276
Epoch 2/20
332/332 [==============================] - 7s 22ms/step - loss: 0.8074 - accuracy: 0.6448 - val_loss: 0.8295 - val_accuracy: 0.6332
Epoch 3/20
332/332 [==============================] - 7s 22ms/step - loss: 0.7756 - accuracy: 0.6625 - val_loss: 0.8063 - val_accuracy: 0.6413
Epoch 4/20
332/332 [==============================] - 6s 18ms/step - loss: 0.7521 - accuracy: 0.6778 - val_loss: 0.8437 - val_accuracy: 0.6286
Epoch 5/20
332/332 [==============================] - 7s 22ms/step - loss: 0.7238 - accuracy: 0.6912 - val_loss: 0.8480 - val_accuracy: 0.6223
Epoch 6/20
267/267 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.62      0.69      0.65      2713
           1       0.58      0.59      0.58      2676
           2       0.68      0.61      0.64      3131

 

In [ ]:
model_lstm = build_model(nb_words, "LSTM", embedding_matrix)
model_lstm.fit(train_X, train_y, epochs=20, batch_size=60,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_lstm.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))


Epoch 1/20
332/332 [==============================] - 6s 9ms/step - loss: 0.8275 - accuracy: 0.6278 - val_loss: 0.7849 - val_accuracy: 0.6559
Epoch 2/20
332/332 [==============================] - 2s 7ms/step - loss: 0.7611 - accuracy: 0.6646 - val_loss: 0.7696 - val_accuracy: 0.6597
Epoch 3/20
332/332 [==============================] - 3s 8ms/step - loss: 0.7302 - accuracy: 0.6801 - val_loss: 0.7492 - val_accuracy: 0.6737
Epoch 4/20
332/332 [==============================] - 3s 9ms/step - loss: 0.7067 - accuracy: 0.6947 - val_loss: 0.7350 - val_accuracy: 0.6746
Epoch 5/20
332/332 [==============================] - 2s 7ms/step - loss: 0.6797 - accuracy: 0.7082 - val_loss: 0.7436 - val_accuracy: 0.6730
Epoch 6/20
332/332 [==============================] - 3s 8ms/step - loss: 0.6495 - accuracy: 0.7223 - val_loss: 0.7461 - val_accuracy: 0.6748
Epoch 7/20
332/332 [==============================] - 3s 8ms/step - loss: 0.6110 - accuracy: 0.7421 - val_loss: 0.7771 - val_accuracy: 0.6695
Epoch 

In [ ]:
model_gru = build_model(nb_words, "GRU", embedding_matrix)
# 20, 120
model_gru.fit(train_X, train_y, epochs=10, batch_size=100,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_gru.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/10
199/199 [==============================] - 5s 13ms/step - loss: 0.8451 - accuracy: 0.6132 - val_loss: 0.7816 - val_accuracy: 0.6554
Epoch 2/10
199/199 [==============================] - 2s 8ms/step - loss: 0.7586 - accuracy: 0.6658 - val_loss: 0.7607 - val_accuracy: 0.6670
Epoch 3/10
199/199 [==============================] - 1s 7ms/step - loss: 0.7338 - accuracy: 0.6802 - val_loss: 0.7473 - val_accuracy: 0.6710
Epoch 4/10
199/199 [==============================] - 1s 7ms/step - loss: 0.7129 - accuracy: 0.6899 - val_loss: 0.7345 - val_accuracy: 0.6813
Epoch 5/10
199/199 [==============================] - 1s 7ms/step - loss: 0.6924 - accuracy: 0.7021 - val_loss: 0.7690 - val_accuracy: 0.6637
Epoch 6/10
199/199 [==============================] - 1s 7ms/step - loss: 0.6670 - accuracy: 0.7135 - val_loss: 0.7367 - val_accuracy: 0.6813
Epoch 7/10
267/267 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.70   

In [ ]:
model_gru.save("rnn.h5")